In [1]:
import torch
import torchvision.models as models
import warnings
import os
import random
import cv2
import faiss
import torch.nn as nn
import matplotlib.pyplot as plt
import albumentations as A
import torch.optim as optim
import numpy as np
import torch.nn.functional as F

from torchvision.models import ResNet50_Weights
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image

In [2]:
from association import *
from detection import Yolo_Nas_L
from utils import draw_bbox_label, make_predicted_video, cal

[2023-07-04 13:47:33] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /opt/ml/sg_logs/console.log


/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-07-04 13:47:37] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-07-04 13:47:37] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-07-04 13:47:37] WARNING - export.py - Failed to import pytorch_quantization
[2023-07-04 13:47:37] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


# Detection 모델 준비

In [3]:
det_weight_path = '/opt/ml/clone/yolo_nas/weight/yolo_nas_l_version4/ckpt_best.pth'
detect_model = Yolo_Nas_L(num_classes=6, checkpoint_path=det_weight_path).cuda()

[2023-07-04 13:47:42] INFO - checkpoint_utils.py - Successfully loaded model weights from /opt/ml/clone/yolo_nas/weight/yolo_nas_l_version4/ckpt_best.pth EMA checkpoint.


weight loaded!!


# Re-ID 모델 준비

In [4]:
#gallery-dataset
gallery_path = '/opt/ml/total/association/gallery' # 1. Gallery 경로!
tf = A.Compose([A.Resize(224,224),
                 A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
gallery_dataset= ETHZ_GALLERY(gallery_path, tf)
gallery_dataset = DataLoader(gallery_dataset, batch_size=128, shuffle=False, num_workers=0, pin_memory=False, drop_last=True)


#re_id_model
re_weight_path = '/opt/ml/total/weight/association/latest.pth' # 2. re_id model weight file 경로!
id_num_classes = 4
re_id_model = ResNet_Triplet(num_classes=id_num_classes).cuda()
re_id_model.load_state_dict(torch.load(os.path.join(re_weight_path)))

<All keys matched successfully>

# Video Inference

In [5]:
video_path = './test_video_short.mp4'
save_path = './predict.mp4'
thr=0.6

In [6]:
make_predicted_video(detect_model, re_id_model, video_path, save_path, gallery_dataset, id_num_classes=id_num_classes, thr=0.5)

100%|██████████| 624/624.0 [1:34:03<00:00,  9.04s/it]
